In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# to check the working directory
os.getcwd()

'C:\\Users\\Dell\\OneDrive - Institute of Business Administration\\Desktop\\IDM Challenge 2'

In [3]:
train = pd.read_excel('Training.xlsx')

In [4]:
test = pd.read_excel('Testing.xlsx')

In [5]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from sklearn import tree
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNetCV

In [6]:
train.head()

,ID,floor,product_type,sub_area,raion_popul,green_zone_part,indust_part,preschool_quota,preschool_education_centers_raion,school_quota,...,market_count_3000,green_part_5000,prom_part_5000,trc_sqm_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,mosque_count_5000,market_count_5000,price_doc
0,1,4.0,Investment,Bibirevo,155572,0.189727,0.000070,5001.0,5,11065.0,...,1.0,13.09,13.31,4036616.0,708.57,1185.71,947.14,1.0,4.0,5850000.0
1,2,2.0,Investment,Tekstil'shhiki,101708,0.112560,0.118537,1463.0,4,5580.0,...,6.0,13.69,21.58,1572990.0,702.68,1196.43,949.55,0.0,10.0,5700000.0
2,3,10.0,Investment,Sokol'niki,57405,0.523439,0.042307,933.0,5,5050.0,...,2.0,25.23,12.76,1656529.0,771.05,1291.18,1031.11,1.0,8.0,5500000.0
3,4,5.0,Investment,Koptevo,96959,0.038456,0.122090,3272.0,4,8751.0,...,6.0,18.73,14.52,1051314.0,711.36,1185.61,948.48,1.0,11.0,5300000.0
4,5,11.0,Investment,Zapadnoe Degunino,78810,0.051844,0.437885,1503.0,3,5782.0,...,1.0,14.86,14.10,1190497.0,714.96,1196.85,955.91,0.0,4.0,4800000.0


In [7]:
train.isna().sum()

ID                                0
floor                          1199
product_type                      0
sub_area                          0
raion_popul                       0
                               ... 
cafe_sum_5000_max_price_avg    2899
cafe_avg_price_5000            2899
mosque_count_5000                 0
market_count_5000                 0
price_doc                         0
Length: 108, dtype: int64

In [9]:
train = train.drop(columns=['sub_area'])
test = test.drop(columns=['sub_area'])
OneHotTrain = pd.get_dummies(train, drop_first=True) #One Hot encoding
OneHotTest = pd.get_dummies(test, drop_first=True)

In [10]:
print(OneHotTrain.shape)
print(OneHotTest.shape)

(100000, 110)
(30000, 109)


In [11]:
Train = OneHotTrain
Test = OneHotTest

In [12]:
Train.fillna(Train.mean(), inplace=True) #Removing null values with mean
Test.fillna(Test.mean(), inplace=True)

In [13]:
Train.isna().sum()

ID                      0
floor                   0
raion_popul             0
green_zone_part         0
indust_part             0
                       ..
railroad_1line_yes      0
ecology_good            0
ecology_no data         0
ecology_poor            0
ecology_satisfactory    0
Length: 110, dtype: int64

In [14]:
grad = GradientBoostingRegressor(max_depth=25, max_features=55, min_samples_split=25,
                      n_estimators=600, random_state=0)
grad.fit(Train.drop(columns=['ID','price_doc']), Train[['price_doc']])
prediction = grad.predict(Test.drop(columns=['ID']))

KeyboardInterrupt: 

In [35]:
rand = RandomForestRegressor(max_depth=30, max_features=50, min_samples_split=20,
                      n_estimators=300, random_state=0)
rand.fit(Train.drop(columns=['ID','price_doc']), Train[['price_doc']])
prediction = rand.predict(Test.drop(columns=['ID']))

In [ ]:
dec = DecisionTreeRegressor(random_state=0)
dec.fit(Train.drop(columns=['ID','price_doc']), Train[['price_doc']])
prediction = dec.predict(Test.drop(columns=['ID']))

In [91]:
reg = LinearRegression().fit(Train.drop(columns=['ID','price_doc']), Train[['price_doc']])
prediction = reg.predict(Test.drop(columns=['ID']))

In [99]:
ex = ExtraTreesRegressor(n_estimators = 800,
          criterion ='mse', max_features = "auto")
prediction = ex.fit(Train.drop(columns=['ID','price_doc']), Train[['price_doc']])
prediction = ex.predict(Test.drop(columns=['ID']))

KeyboardInterrupt: 

In [13]:
x = Train[['floor','raion_popul', 'green_zone_part', 'indust_part', 'preschool_quota', 'school_quota']]
y = Train[['price_doc']]
poly = PolynomialFeatures(5, interaction_only=True)
X2 = poly.fit_transform(x)
t = Test[['floor','raion_popul', 'green_zone_part', 'indust_part', 'preschool_quota', 'school_quota']]
T2 = poly.fit_transform(t)
poly_reg = GradientBoostingRegressor(max_depth=25, max_features=50, min_samples_split=20,
                      n_estimators=500, random_state=0).fit(X2, y)
prediction = poly_reg.predict(T2)

,floor,raion_popul,green_zone_part,indust_part,preschool_quota,preschool_education_centers_raion,school_quota,school_education_centers_top_20_raion,healthcare_centers_raion,university_top_20_raion,...,big_market_raion_yes,nuclear_reactor_raion_yes,detention_facility_raion_yes,water_1line_yes,big_road1_1line_yes,railroad_1line_yes,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
0,4.000000,155572,0.189727,0.000070,5001.000000,5,11065.0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,2.000000,101708,0.112560,0.118537,1463.000000,4,5580.0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,10.000000,57405,0.523439,0.042307,933.000000,5,5050.0,0,3,0,...,0,0,1,0,0,0,0,0,1,0
3,5.000000,96959,0.038456,0.122090,3272.000000,4,8751.0,0,4,0,...,0,0,0,0,0,0,0,0,1,0
4,11.000000,78810,0.051844,0.437885,1503.000000,3,5782.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,6.781051,3521,0.004676,0.000000,2487.148369,0,1012.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
99996,9.641840,3521,0.004676,0.000000,2487.148369,0,1012.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
99997,6.956026,3521,0.004676,0.000000,2487.148369,0,1012.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
99998,7.704027,3521,0.004676,0.000000,2487.148369,0,1012.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [24]:
x = Train.drop(columns=['ID','price_doc'])
y = Train[['price_doc']]

reg = LinearRegression()
sfs = SequentialFeatureSelector(reg, direction='backward',n_features_to_select=106).fit(x, y)
print(sfs.get_feature_names_out())

['floor' 'raion_popul' 'green_zone_part' 'indust_part' 'preschool_quota'
 'preschool_education_centers_raion' 'school_quota'
 'school_education_centers_top_20_raion' 'healthcare_centers_raion'
 'university_top_20_raion' 'sport_objects_raion'
 'additional_education_raion' 'shopping_centers_raion'
 'raion_build_count_with_material_info' 'build_count_brick'
 'build_count_monolith' 'build_count_panel' 'build_count_foam'
 'build_count_before_1920' 'build_count_1946-1970' 'build_count_1971-1995'
 'ID_metro' 'metro_min_walk' 'green_zone_km' 'industrial_km'
 'water_treatment_km' 'cemetery_km' 'railroad_station_walk_km'
 'ID_railroad_station_walk' 'ID_railroad_station_avto' 'water_km'
 'big_road1_km' 'ID_big_road1' 'big_road2_km' 'ID_big_road2'
 'ID_railroad_terminal' 'ID_bus_terminal' 'oil_chemistry_km'
 'big_market_km' 'office_km' 'church_synagogue_km' 'green_part_500'
 'prom_part_500' 'office_sqm_500' 'trc_count_500' 'sport_count_500'
 'market_count_500' 'green_part_1000' 'trc_count_1000' 't

In [47]:
x25 = Train[['floor', 'indust_part', 'preschool_quota',
 'preschool_education_centers_raion', 'school_quota',
 'university_top_20_raion', 'sport_objects_raion',
 'raion_build_count_with_material_info', 'build_count_brick',
 'build_count_monolith', 'build_count_panel', 'ID_metro', 'metro_min_walk',
 'big_road2_km', 'prom_part_1500', 'cafe_sum_3000_max_price_avg',
 'mosque_count_3000', 'sport_count_3000', 'mosque_count_5000',
 'product_type_OwnerOccupier', 'thermal_power_plant_raion_yes',
 'incineration_raion_yes', 'railroad_terminal_raion_yes',
 'railroad_1line_yes', 'ecology_good']]
t25 = Test[['floor', 'indust_part', 'preschool_quota',
 'preschool_education_centers_raion', 'school_quota',
 'university_top_20_raion', 'sport_objects_raion',
 'raion_build_count_with_material_info', 'build_count_brick',
 'build_count_monolith', 'build_count_panel', 'ID_metro', 'metro_min_walk',
 'big_road2_km', 'prom_part_1500', 'cafe_sum_3000_max_price_avg',
 'mosque_count_3000', 'sport_count_3000', 'mosque_count_5000',
 'product_type_OwnerOccupier', 'thermal_power_plant_raion_yes',
 'incineration_raion_yes', 'railroad_terminal_raion_yes',
 'railroad_1line_yes', 'ecology_good']]

In [48]:
xx = Train[['floor', 'raion_popul', 'green_zone_part', 'indust_part', 'preschool_quota',
 'preschool_education_centers_raion', 'school_quota',
 'school_education_centers_top_20_raion', 'healthcare_centers_raion',
 'university_top_20_raion', 'sport_objects_raion',
 'additional_education_raion', 'shopping_centers_raion',
 'raion_build_count_with_material_info', 'build_count_brick',
 'build_count_monolith', 'build_count_panel', 'build_count_foam',
 'build_count_before_1920', 'build_count_1946-1970', 'build_count_1971-1995',
 'ID_metro', 'metro_min_walk', 'green_zone_km', 'industrial_km',
 'water_treatment_km', 'cemetery_km', 'railroad_station_walk_km',
 'ID_railroad_station_walk', 'ID_railroad_station_avto', 'water_km',
 'big_road1_km', 'ID_big_road1', 'big_road2_km', 'ID_big_road2',
 'ID_railroad_terminal', 'ID_bus_terminal', 'oil_chemistry_km',
 'big_market_km', 'office_km', 'church_synagogue_km', 'green_part_500',
 'prom_part_500', 'office_sqm_500', 'trc_count_500', 'sport_count_500',
 'market_count_500', 'green_part_1000', 'trc_count_1000', 'trc_sqm_1000',
 'cafe_sum_1000_min_price_avg', 'cafe_sum_1000_max_price_avg',
 'cafe_avg_price_1000', 'mosque_count_1000', 'leisure_count_1000',
 'sport_count_1000', 'market_count_1000', 'prom_part_1500',
 'office_count_1500', 'trc_count_1500', 'trc_sqm_1500',
 'cafe_sum_1500_min_price_avg', 'cafe_sum_1500_max_price_avg',
 'cafe_avg_price_1500', 'sport_count_1500', 'market_count_1500',
 'green_part_2000', 'trc_sqm_2000', 'cafe_sum_2000_min_price_avg',
 'cafe_sum_2000_max_price_avg', 'cafe_avg_price_2000', 'mosque_count_2000',
 'prom_part_3000', 'trc_count_3000', 'trc_sqm_3000',
 'cafe_sum_3000_min_price_avg', 'cafe_sum_3000_max_price_avg',
 'cafe_avg_price_3000', 'mosque_count_3000', 'sport_count_3000',
 'market_count_3000', 'green_part_5000', 'prom_part_5000', 'trc_sqm_5000',
 'cafe_sum_5000_min_price_avg', 'cafe_sum_5000_max_price_avg',
 'cafe_avg_price_5000', 'mosque_count_5000', 'market_count_5000',
 'product_type_OwnerOccupier', 'culture_objects_top_25_yes',
 'thermal_power_plant_raion_yes', 'incineration_raion_yes',
 'oil_chemistry_raion_yes', 'radiation_raion_yes',
 'railroad_terminal_raion_yes', 'big_market_raion_yes',
 'nuclear_reactor_raion_yes', 'detention_facility_raion_yes',
 'water_1line_yes', 'big_road1_1line_yes', 'railroad_1line_yes',
 'ecology_good', 'ecology_no data', 'ecology_poor', 'ecology_satisfactory']]
tt = Test[['floor', 'raion_popul', 'green_zone_part', 'indust_part', 'preschool_quota',
 'preschool_education_centers_raion', 'school_quota',
 'school_education_centers_top_20_raion', 'healthcare_centers_raion',
 'university_top_20_raion', 'sport_objects_raion',
 'additional_education_raion', 'shopping_centers_raion',
 'raion_build_count_with_material_info', 'build_count_brick',
 'build_count_monolith', 'build_count_panel', 'build_count_foam',
 'build_count_before_1920', 'build_count_1946-1970', 'build_count_1971-1995',
 'ID_metro', 'metro_min_walk', 'green_zone_km', 'industrial_km',
 'water_treatment_km', 'cemetery_km', 'railroad_station_walk_km',
 'ID_railroad_station_walk', 'ID_railroad_station_avto', 'water_km',
 'big_road1_km', 'ID_big_road1', 'big_road2_km', 'ID_big_road2',
 'ID_railroad_terminal', 'ID_bus_terminal', 'oil_chemistry_km',
 'big_market_km', 'office_km', 'church_synagogue_km', 'green_part_500',
 'prom_part_500', 'office_sqm_500', 'trc_count_500', 'sport_count_500',
 'market_count_500', 'green_part_1000', 'trc_count_1000', 'trc_sqm_1000',
 'cafe_sum_1000_min_price_avg', 'cafe_sum_1000_max_price_avg',
 'cafe_avg_price_1000', 'mosque_count_1000', 'leisure_count_1000',
 'sport_count_1000', 'market_count_1000', 'prom_part_1500',
 'office_count_1500', 'trc_count_1500', 'trc_sqm_1500',
 'cafe_sum_1500_min_price_avg', 'cafe_sum_1500_max_price_avg',
 'cafe_avg_price_1500', 'sport_count_1500', 'market_count_1500',
 'green_part_2000', 'trc_sqm_2000', 'cafe_sum_2000_min_price_avg',
 'cafe_sum_2000_max_price_avg', 'cafe_avg_price_2000', 'mosque_count_2000',
 'prom_part_3000', 'trc_count_3000', 'trc_sqm_3000',
 'cafe_sum_3000_min_price_avg', 'cafe_sum_3000_max_price_avg',
 'cafe_avg_price_3000', 'mosque_count_3000', 'sport_count_3000',
 'market_count_3000', 'green_part_5000', 'prom_part_5000', 'trc_sqm_5000',
 'cafe_sum_5000_min_price_avg', 'cafe_sum_5000_max_price_avg',
 'cafe_avg_price_5000', 'mosque_count_5000', 'market_count_5000',
 'product_type_OwnerOccupier', 'culture_objects_top_25_yes',
 'thermal_power_plant_raion_yes', 'incineration_raion_yes',
 'oil_chemistry_raion_yes', 'radiation_raion_yes',
 'railroad_terminal_raion_yes', 'big_market_raion_yes',
 'nuclear_reactor_raion_yes', 'detention_facility_raion_yes',
 'water_1line_yes', 'big_road1_1line_yes', 'railroad_1line_yes',
 'ecology_good', 'ecology_no data', 'ecology_poor', 'ecology_satisfactory']]

In [37]:
y = Train[['price_doc']]
reg = LinearRegression().fit(xx, y)
prediction = reg.predict(tt)

In [16]:
y = Train[['price_doc']]
poly = PolynomialFeatures(2, interaction_only=False)
X2 = poly.fit_transform(x25)
T2 = poly.fit_transform(t25)
poly_reg = GradientBoostingRegressor(max_depth=20, max_features=50, min_samples_split=20,
                      n_estimators=500, random_state=0).fit(X2, y)
prediction = poly_reg.predict(T2)

In [57]:
y = Train[['price_doc']] #for backwarding
poly = PolynomialFeatures(2, interaction_only=False)
X2 = poly.fit_transform(xx)
T2 = poly.fit_transform(tt)
poly_reg = GradientBoostingRegressor(max_depth=15, max_features=25, min_samples_split=15,
                      n_estimators=500, random_state=0).fit(X2, y)
prediction = poly_reg.predict(T2)

In [58]:
prediction = pd.DataFrame(prediction, columns=['price_doc'])

In [59]:
prediction

,price_doc
0,5.932450e+06
1,8.288456e+06
2,1.785805e+07
3,5.497550e+06
4,8.091359e+06
...,...
29995,4.059050e+06
29996,1.770799e+06
29997,4.845582e+06
29998,1.241758e+06


In [60]:
output = pd.read_csv('sample.csv')

In [61]:
output

,ID,price_doc
0,100001,100000
1,100002,100000
2,100003,100000
3,100004,100000
4,100005,100000
...,...,...
29995,129996,100000
29996,129997,100000
29997,129998,100000
29998,129999,100000


In [62]:
output['price_doc'] = prediction

In [63]:
output

,ID,price_doc
0,100001,5.932450e+06
1,100002,8.288456e+06
2,100003,1.785805e+07
3,100004,5.497550e+06
4,100005,8.091359e+06
...,...,...
29995,129996,4.059050e+06
29996,129997,1.770799e+06
29997,129998,4.845582e+06
29998,129999,1.241758e+06


In [64]:
output.to_csv('output.csv', index = False)